# North Atlantic O2 gapfill projection
    - Uses pre-computed ML model
    - Absolute magnitude of O2 is estimated as a function of T, S, long, lat, and time (year, month)
    - Separates the data into training and test data (75% - 25%)
    - calculate for each z level, native in EN4 grid

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import sklearn as skl
import gsw
import cartopy.crs as ccrs
from scipy.interpolate import interp1d
import os
import warnings
warnings.filterwarnings('ignore')
import joblib
from multiprocessing import Pool

In [2]:
# version information
ver = '1.3'
# select ML model
modname='NN'
# 
# zoffset
zoff=100

In [3]:
# observational data 
# -------------------------------------------------------------
# Needs adjustment: this is where you write out the output files
diro = '/glade/scratch/ito/WOD18_OSDCTD/'
# -------------------------------------------------------------
dirf = '/glade/campaign/univ/ugit0034/EN4/L09_20x180x360/'
dirin = '/glade/campaign/univ/ugit0034/WOD18_OSDCTD/'
fosd='_1x1bin_osd_'
fctd='_1x1bin_ctd_'
fmer='_1x1bin_merged_'
var=['o2','TSN2']
os.system('mkdir -p '+diro)
os.system('mkdir -p '+diro+'temp')

0

In [4]:
# obtain vertical grid
ds=xr.open_dataset(dirin+var[0]+fmer+str(1965)+'.nc')
Z=ds.depth.to_numpy()
Nz=np.size(Z)

In [5]:
# select analysis period
# do not change the start year from 1965 (this is when Carpenter 1965 established modern Winkler method)
yrs=np.arange(1965,2011,1)
t=np.arange('1965-01','2011-01',dtype='datetime64[M]')

In [6]:
MLmodel = joblib.load(f'{modname}_model_v{ver}.sav')
# read in additional parameters
params = np.load(f'ML_params_v{ver}.npz')
Xm=params['Xm']
Xstd=params['Xstd']
ym=params['ym']
ystd=params['ystd']

In [7]:
# get mld clim
mldc=np.load('mldc.npy')

In [8]:
dwoa=xr.open_dataset('/glade/campaign/univ/ugit0034/woa18/woa18_all_o00_01.nc',decode_times=False)

In [9]:
# basin mask
dsm=xr.open_dataset('/glade/campaign/univ/ugit0034/wod18/basin_mask_01.nc')
#! cp /glade/work/ito/basin_mask_01.nc /glade/campaign/univ/ugit0034/wod18/.

In [10]:
# select Atlantic basin
# Atlantic is mask = 1
ma = dsm.basin_mask.sel(depth=Z).to_numpy()

In [11]:
zlev=300
kind=[idx for idx,elem in enumerate(Z) if elem==zlev]
maz=np.squeeze(ma[kind,:,:])
#
mon=["%.2d" % i for i in np.arange(1,13,1)]
#
dc=xr.open_dataset(dirf+'EN4_TSN2_L09_180x360_'+str(1965)+mon[0]+'.nc')
y=dc.lat.to_numpy()
x=dc.lon.to_numpy()
Ny=np.size(y)
Nx=np.size(x)
Nt=np.size(yrs)*12
xx,yy=np.meshgrid(x,y)
#
depth1 = dc.depth.to_numpy()
Nz1 = np.size(depth1)

In [12]:
# apply basin mask 
def apply_basinmask(datain):
    dataout=np.where((maz==1)&(yy>0),datain,np.nan)
    return dataout

In [13]:
def moving_average(data,w):
    weight=np.ones(w)/w
    smooth_data=np.convolve(data, weight, mode='same')
    return smooth_data

In [14]:
# get input data from full model
def get_inputdata(zlev,it,year,mn):
    #dc = xr.open_dataset(dirf+'EN4_TSN2_G10_180x360_'+str(year)+mon[mn]+'.nc')
    dc = xr.open_dataset(dirf+'EN4_TSN2_L09_180x360_'+str(year)+mon[mn]+'.nc')
    soa=dc.SA.interp(depth=zlev).to_numpy().squeeze()
    toa=dc.CT.interp(depth=zlev).to_numpy().squeeze()
    mld=np.squeeze(mldc[mn,:,:])
    return soa,toa,mld

In [15]:
# generate data matrix
def gen_datamatrix(xi,yi,it,x1,x2,x3,x4,mld):
    X1 = x1.flatten() # 
    X2 = x2.flatten() # 
    X3 = x3.flatten() # 
    X4 = x4.flatten() # 
    tt0  = np.ones((Ny,Nx))*it
    X5 = tt0.flatten() # decimal year 
    X6 = X5%12         # month
    xxi = xi.flatten() # lon
    yyi = yi.flatten() # lat
    # 
    ml1 = mld.flatten()
    X6 = np.where(ml1>zlev-zoff,X6,2)
    # remove nan
    #print([np.size(X1),np.size(X2),np.size(X3),np.size(X4),np.size(X5)])
    dd = X1+X2+X3+X4+X5
    X11=X1[np.isnan(dd)==False]
    X21=X2[np.isnan(dd)==False]
    X31=X3[np.isnan(dd)==False]
    X41=X4[np.isnan(dd)==False]
    X51=X5[np.isnan(dd)==False]
    X61=X6[np.isnan(dd)==False]
    #
    Xi=xxi[np.isnan(dd)==False]
    Yi=yyi[np.isnan(dd)==False]
    #
    zin = np.ones(np.size(X11))*zlev
    # Normalize data
    # generate data matrix and standardize it
    X = np.array([X11, X21, X31, X41, X51, X61])
    #X = np.array([X11, X21, X31, X41, zin, X51, X61])
    Xa = (X.T - Xm)/Xstd
    Nsample = np.size(X11)
    #print(Nsample)
    return Xa,Xi,Yi

In [16]:
def map_yearly(year):
    Nx=np.size(x)
    Ny=np.size(y)
    zlev_arr=np.array([zlev])
    o2est2=np.zeros((12,1,Ny,Nx))
    xxi,yyi=np.meshgrid(np.arange(0,Nx,1),np.arange(0,Ny,1))
    if year%10 == 5:
        print('year = '+str(year))
    t=np.arange(str(year)+'-01',str(year+1)+'-01',dtype='datetime64[M]')
    for month in range(12):
        it = month+(year-1965)*12
        soa,toa,mld = get_inputdata(zlev,it,year,month)
        # apply mask
        soa=apply_basinmask(soa)
        toa=apply_basinmask(toa)
        mld=apply_basinmask(mld)
        # generate data matrix
        Xa,xi,yi=gen_datamatrix(xxi,yyi,it,soa,toa,xx,yy,mld)
        temp = np.shape(Xa)
        Nsample=temp[0]
        # projection
        out = reg.predict(Xa)
        # map it back to lon-lat grid
        temp = np.nan*np.zeros((Ny,Nx))
        for n in range(Nsample):
            temp[yi[n],xi[n]]=out[n]
        o2est2[month,0,:,:] = temp*ystd + ym
    da1=xr.DataArray(data=o2est2,name='o2est',dims=['time','depth','lat','lon'],
                 coords={'time':t,'depth':zlev_arr,'lat':y,'lon':x})
    ds=da1.to_dataset()
    ds.to_netcdf(diro+'temp/o2est_'+str(year)+'.nc')
    return 0

In [17]:
zlevels = depth1
#
# reconstruction in parallel mode
#
reg=MLmodel
x=dc.lon
y=dc.lat
#
for zlev_cnt,zlev in enumerate(zlevels):
    print(f'calculating {zlev}m')
    maz = dsm.basin_mask.interp(depth=zlev).to_numpy()
    #kind=[idx for idx,elem in enumerate(Z) if elem==zlev]
    #maz=np.squeeze(ma[kind,:,:])
    os.system('rm '+diro+'/temp/*.nc')
    #
    if __name__ == '__main__':
        with Pool(10) as p:
            print(p.map(map_yearly, yrs))
    #
    # save the result as a netCDF file
    #
    dtemp=xr.open_mfdataset(diro+'temp/o2est*.nc')
    dtemp.to_netcdf(diro+'/O2abs_NA_'+modname+'_WOD18O2xEN4TS_monthly_v'+ver+'_z'+str(int(zlev))+'.nc')

calculating 6.0m
year = 1965year = 1975

year = 1985
year = 1995
year = 2005
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
calculating 10.0m
year = 1965year = 1975

year = 1985
year = 1995
year = 2005
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
calculating 20.0m
year = 1965year = 1975

year = 1985
year = 1995
year = 2005
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
calculating 30.0m
year = 1965year = 1975

year = 1985
year = 1995
year = 2005
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
calculating 40.0m
year = 1965year = 1975

year = 1985
year = 1995
year = 2005
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [18]:
#! rm /glade/scratch/ito/WOD18_OSDCTD/O2abs_NA_RF_*v1.3*.nc

In [19]:
ds=xr.open_mfdataset(f'{diro}O2abs_NA_{modname}*')

In [20]:
os.system(f'rm {diro}O2abs_NA_{modname}_WOD18O2xEN4TS_monthly_v{ver}.nc')
ds.to_netcdf(f'{diro}O2abs_NA_{modname}_WOD18O2xEN4TS_monthly_v{ver}.nc')

In [21]:
os.system(f'ls -althr {diro}O2abs_NA_{modname}*')

-rw-r--r-- 1 ito ncar 297M Aug  1 18:33 /glade/scratch/ito/WOD18_OSDCTD/O2abs_NA_NN_WOD18O2xEN4TS_monthly_v1.1_z6.nc
-rw-r--r-- 1 ito ncar 297M Aug  1 18:34 /glade/scratch/ito/WOD18_OSDCTD/O2abs_NA_NN_WOD18O2xEN4TS_monthly_v1.1_z10.nc
-rw-r--r-- 1 ito ncar 297M Aug  1 18:34 /glade/scratch/ito/WOD18_OSDCTD/O2abs_NA_NN_WOD18O2xEN4TS_monthly_v1.1_z20.nc
-rw-r--r-- 1 ito ncar 297M Aug  1 18:34 /glade/scratch/ito/WOD18_OSDCTD/O2abs_NA_NN_WOD18O2xEN4TS_monthly_v1.1_z30.nc
-rw-r--r-- 1 ito ncar 297M Aug  1 18:34 /glade/scratch/ito/WOD18_OSDCTD/O2abs_NA_NN_WOD18O2xEN4TS_monthly_v1.1_z40.nc
-rw-r--r-- 1 ito ncar 297M Aug  1 18:34 /glade/scratch/ito/WOD18_OSDCTD/O2abs_NA_NN_WOD18O2xEN4TS_monthly_v1.1_z50.nc
-rw-r--r-- 1 ito ncar 297M Aug  1 18:34 /glade/scratch/ito/WOD18_OSDCTD/O2abs_NA_NN_WOD18O2xEN4TS_monthly_v1.1_z65.nc
-rw-r--r-- 1 ito ncar 297M Aug  1 18:34 /glade/scratch/ito/WOD18_OSDCTD/O2abs_NA_NN_WOD18O2xEN4TS_monthly_v1.1_z80.nc
-rw-r--r-- 1 ito ncar 297M Aug  1 18:35 /glade/scratch/it

0